In [1]:
from detector import FaceDetector
import cv2
import av
from IPython.display import Video

In [ ]:
face_detector = FaceDetector(model_path='best.onnx',
                             crops_path="crops/",
                             meta_path="meta/meta.json")

input_filename = "assets/train_station.mp4"
output_filename = "inference_train_station.mp4"

input_container = av.open(input_filename)
output_container = av.open(output_filename, "w")
stream = output_container.add_stream("h264")

frame_count = 0
# Запись изображений в видео
for frame in input_container.decode(video=0):
    if frame_count > 500:
        break
    # Получаем изображение кадра
    image = frame.to_ndarray(format='rgb24')

    # Обрабатываем кадр с помощью детектора лиц
    results = face_detector.update(image)
    annotated_frame = results[0].plot()
    frame = av.VideoFrame.from_ndarray(annotated_frame, format="rgb24")
    packet = stream.encode(frame)
    output_container.mux(packet)
    frame_count += 1
# Завершение записи видео
packet = stream.encode(None)  # Посылаем None, чтобы завершить видеопоток
output_container.mux(packet)
# Закрытие видеопотока
output_container.close()
input_container.close()

print(f"Видео сгенерировано: {output_filename}")

In [3]:
Video("inference_train_station.mp4", width=990, height=360)